 Phone PE pulse 

In [4]:
# !git clone https://github.com/PhonePe/pulse.git #dataset cloned from github using 'git clone'

In [1]:
pip install numpy pandas sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [3]:
# verifying using special variable
import psycopg2
print(psycopg2.__version__)

import numpy
print(numpy.__version__)  

import pandas
print(pandas.__version__)  

import sqlalchemy
print(sqlalchemy.__version__)

2.9.10 (dt dec pq3 ext lo64)
2.3.1
2.3.1
2.0.41


In [3]:
import os #'OS' module for System operations like file handling
import json
import pandas as pd

In [2]:
# ✅ Step 0: Set the root path to the 'state' folder
root_path = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/pulse/data/aggregated/insurance/country/india/state'

Looping into folders & extracts the data

In [ ]:
# Make a list to store all the data
data_rows = []

# Go through each state folder
for state in os.listdir(root_path):
    state_path = os.path.join(root_path, state)
    if os.path.isdir(state_path):  # Only look at folders
        # Go through each year folder
        for year in os.listdir(state_path):
            year_path = os.path.join(state_path, year)
            if os.path.isdir(year_path):
                # Go through each quarter file
                for file in os.listdir(year_path):
                    if file.endswith(".json"):
                        file_path = os.path.join(year_path, file)
                        quarter = int(file.replace(".json", ""))
                        try:
                            # Open and read the JSON file
                            with open(file_path, "r") as f:
                                data = json.load(f)
                            # Get the list of transactions
                            txns = data.get("data", {}).get("transactionData", [])
                            # Find the insurance info
                            ins = next((x for x in txns if x.get("name") == "Insurance"), None)
                            # If no insurance, add empty info
                            if not ins:
                                data_rows.append({
                                    "state": state,
                                    "year": int(year),
                                    "quarter": quarter,
                                    "insurance_type": None,
                                    "transaction_count": None,
                                    "amount": None
                                })
                            else:
                                # Add info for each payment type
                                for pay in ins.get("paymentInstruments", []):
                                    data_rows.append({
                                        "state": state,
                                        "year": int(year),
                                        "quarter": quarter,
                                        "insurance_type": pay.get("type"),
                                        "transaction_count": pay.get("count"),
                                        "amount": pay.get("amount")
                                    })
                        except Exception as e:
                            print(f"⚠️ Skipped {file_path}: {e}")

# Make a DataFrame from the list
df = pd.DataFrame(data_rows)

# Remove rows with missing important info
df = df.dropna(subset=['transaction_count', 'amount', 'year', 'quarter'])

# Save the table as a CSV file
output_folder = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv"
output_csv_path = os.path.join(output_folder, "aggregated_insurance.csv")
os.makedirs(output_folder, exist_ok=True)
df.to_csv(output_csv_path, index=False)
print(f"✅ CSV saved at: {output_csv_path}")


✅ CSV saved at: /Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/aggregated_insurance.csv


inserting into SQL

In [6]:
import psycopg2
import pandas as pd

# Step 1: Load CSV
csv_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/aggregated_insurance.csv"
df = pd.read_csv(csv_path)

# Step 2: Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()
print("✅ Connected to PostgreSQL.")

# Step 3: Create table if not exists
create_table_query = """
CREATE TABLE IF NOT EXISTS aggregated_insurance (
    state TEXT,
    year INTEGER,
    quarter INTEGER,
    insurance_type TEXT,
    transaction_count BIGINT,
    amount DOUBLE PRECISION
);
"""
cur.execute(create_table_query)
conn.commit()

# Step 4: Insert rows
insert_query = """
INSERT INTO aggregated_insurance (state, year, quarter, insurance_type, transaction_count, amount)
VALUES (%s, %s, %s, %s, %s, %s)
"""

for _, row in df.iterrows():
    try:
        cur.execute(insert_query, tuple(row))
    except Exception as e:
        print("❌ Error row:", row)
        print("❌ Error:", e)
        break  # stop on first error


# Step 5: Close connection
cur.close()
conn.close()


✅ Connected to PostgreSQL.


Aggregated Transcations

In [7]:
import os
import json
import pandas as pd
from pathlib import Path

# === 🔧 CONFIGURATION ===
BASE_PATH = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/pulse/data/aggregated/transaction/country/india/state'
OUTPUT_CSV = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/aggregated_transactions.csv'

# === 📥 Data Collector ===
data_rows = []

# Walk through all state folders
for state_folder in os.listdir(BASE_PATH):
    state_path = os.path.join(BASE_PATH, state_folder)
    if not os.path.isdir(state_path):
        continue

    for year_folder in os.listdir(state_path):
        year_path = os.path.join(state_path, year_folder)
        if not os.path.isdir(year_path):
            continue

        for quarter_file in os.listdir(year_path):
            if not quarter_file.endswith('.json'):
                continue

            quarter = int(quarter_file.replace('Q', '').replace('.json', ''))  # e.g., Q1.json → 1
            file_path = os.path.join(year_path, quarter_file)

            with open(file_path, 'r') as f:
                try:
                    content = json.load(f)
                except json.JSONDecodeError:
                    print(f"❌ Invalid JSON in {file_path}")
                    continue

            data = content.get("data", {})
            transaction_data = data.get("transactionData", [])
            from_ts = data.get("from")
            to_ts = data.get("to")
            response_ts = content.get("responseTimestamp")

            if not from_ts or not to_ts:
                continue

            for txn_entry in transaction_data:
                category = txn_entry.get("name")
                instruments = txn_entry.get("paymentInstruments", [])

                for instrument in instruments:
                    row = {
                        "state": state_folder,
                        "year": int(year_folder),
                        "quarter": quarter,
                        "category": category,
                        "txn_type": instrument.get("type"),
                        "txn_count": instrument.get("count"),
                        "txn_amount": instrument.get("amount"),                       
                    }
                    data_rows.append(row)

# === 💾 Save to CSV for PostgreSQL ===
if not data_rows:
    print("⚠️ No data found!")
else:
    df = pd.DataFrame(data_rows)

    # Reorder columns to match PostgreSQL table
    df = df[
        [
            "state", "year", "quarter", "category",
            "txn_type", "txn_count", "txn_amount",
        ]
    ]

    # Convert all fields to string to avoid type mismatches
    df = df.astype(str)

    # Ensure output folder exists
    output_dir = Path(OUTPUT_CSV).parent
    output_dir.mkdir(parents=True, exist_ok=True)

    df.to_csv(OUTPUT_CSV, index=False, header=True)  
    print(f"✅ Cleaned CSV (PostgreSQL-ready) exported to:\n{OUTPUT_CSV}")


✅ Cleaned CSV (PostgreSQL-ready) exported to:
/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/aggregated_transactions.csv


Connect to PostgreSQL

In [12]:
import psycopg2
import csv

# === 🛠 PostgreSQL Connection Config ===
conn = None
try:
    conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="1234",
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()
    print("✅ Connected to PostgreSQL.")

    # === 📦 Create table if it doesn't exist
    cur.execute("""
        CREATE TABLE IF NOT EXISTS aggregated_transactions (
            id SERIAL PRIMARY KEY,
            state TEXT,
            year INTEGER,
            quarter INTEGER,
            category TEXT,
            txn_type TEXT,
            txn_count BIGINT,
            txn_amount DOUBLE PRECISION
        );
    """)
    conn.commit()
    print("🧱 Table checked/created.")

    # === 📤 Insert from CSV
    with open('/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/aggregated_transactions.csv', 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            cur.execute("""
                INSERT INTO aggregated_transactions (
                    state, year, quarter,
                    category, txn_type,
                    txn_count, txn_amount
                ) VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, (
                row['state'],
                int(row['year']),
                int(row['quarter']),
                row['category'],
                row['txn_type'],
                int(row['txn_count']) if row['txn_count'] else 0,
                float(row['txn_amount']) if row['txn_amount'] else 0.0,
            ))

    conn.commit()
    print("📥 Data inserted successfully.")

except Exception as e:
    print("❌ Error during DB operation:", e)

finally:
    if conn:
        conn.close()
        print("🔒 Connection closed.")


✅ Connected to PostgreSQL.
🧱 Table checked/created.
📥 Data inserted successfully.
🔒 Connection closed.


Aggregated Users

In [37]:
import os
import json
import csv

# Paths
input_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/pulse/data/aggregated/user/country/india/state"
output_csv = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/aggregated_user.csv"

# Output container
final_data = []

def safe_extract(json_data, key, default=None):
    """Safely extract nested values."""
    return json_data.get('data', {}).get(key, default)

# Traverse all states
for state in os.listdir(input_path):
    state_path = os.path.join(input_path, state)
    if not os.path.isdir(state_path):
        continue

    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)

        for file in os.listdir(year_path):
            if file.endswith('.json'):
                quarter = file.replace('.json', '')
                file_path = os.path.join(year_path, file)

                try:
                    with open(file_path, 'r') as f:
                        data = json.load(f)

                    aggregated = safe_extract(data, 'aggregated', {})
                    registered_users = aggregated.get('registeredUsers', 0)
                    app_opens = aggregated.get('appOpens', 0)

                    devices = safe_extract(data, 'usersByDevice', [])

                    if devices:
                        for device in devices:
                            row = {
                                'state': state,
                                'year': int(year),
                                'quarter': int(quarter),
                                'registered_users': registered_users,
                                'app_opens': app_opens,
                                'device_brand': device.get('brand'),
                                'device_count': device.get('count'),
                                'device_percentage': device.get('percentage'),
                            }
                            final_data.append(row)
                    else:
                        # Still log row even if usersByDevice is None or empty
                        row = {
                            'state': state,
                            'year': int(year),
                            'quarter': int(quarter),
                            'registered_users': registered_users,
                            'app_opens': app_opens,
                            'device_brand': None,
                            'device_count': None,
                            'device_percentage': None,
                        }
                        final_data.append(row)

                except Exception as e:
                    print(f"⚠️ Error reading {file_path}: {e}")

# Write CSV
with open(output_csv, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=[
        'state', 'year', 'quarter',
        'registered_users', 'app_opens',
        'device_brand', 'device_count', 'device_percentage'
    ])
    writer.writeheader()
    writer.writerows(final_data)

print("✅ CSV file 'aggregated_user.csv' generated successfully.")


✅ CSV file 'aggregated_user.csv' generated successfully.


Invert into DB

In [19]:
import psycopg2
import csv

# PostgreSQL connection
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Step 1: Create table
cur.execute("""
    DROP TABLE IF EXISTS aggregated_user;

CREATE TABLE IF NOT EXISTS aggregated_user (
    id SERIAL PRIMARY KEY,
    state TEXT,
    year INT,
    quarter INT,
    registered_users BIGINT,
    app_opens BIGINT,
    device_brand TEXT,
    device_count BIGINT,
    device_percentage FLOAT
);
""")
conn.commit()

# Step 2: Insert data from CSV
with open('/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/aggregated_user.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        cur.execute("""
            INSERT INTO aggregated_user (
                state, year, quarter,
                registered_users, app_opens,
                device_brand, device_count, device_percentage
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
        """, (
            row['state'],
            int(row['year']),
            int(row['quarter']),
            int(row['registered_users']) if row['registered_users'] else 0,
            int(row['app_opens']) if row['app_opens'] else 0,
            row['device_brand'],
            int(row['device_count']) if row['device_count'] else None,
            float(row['device_percentage']) if row['device_percentage'] else None,
        ))

conn.commit()
cur.close()
conn.close()

print("✅ Data inserted into PostgreSQL table 'aggregated_user'.")


✅ Data inserted into PostgreSQL table 'aggregated_user'.


Map - insurance - Country/india

In [40]:
import os
import json
import pandas as pd

base_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/pulse/data/map/insurance/hover/country/india/state"
output_csv_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/insurance_hover.csv"

hover_data_rows = []

for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    if not os.path.isdir(state_path):
        continue

    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        if not os.path.isdir(year_path):
            continue

        for file in os.listdir(year_path):
            if not file.endswith(".json"):
                continue

            quarter = file.replace(".json", "")
            file_path = os.path.join(year_path, file)

            try:
                with open(file_path, 'r') as f:
                    content = json.load(f)

                hover_list = content.get("data", {}).get("hoverDataList", [])

                for item in hover_list:
                    district = item.get("name")
                    metrics = item.get("metric", [])

                    for metric in metrics:
                        hover_data_rows.append({
                            "state_name": state,
                            "year": int(year),
                            "quarter": int(quarter),
                            "district_name": district,
                            "metric_type": metric.get("type"),
                            "policy_count": metric.get("count"),
                            "amount": metric.get("amount")
                        })

            except Exception as e:
                print(f"❌ Failed to parse: {file_path} → {e}")

# Create DataFrame
df = pd.DataFrame(hover_data_rows)

# Save to CSV
df.to_csv(output_csv_path, index=False)
print(f"✅ CSV saved to: {output_csv_path}")
print("📊 Total rows written:", len(df))

# Optional: Show null counts
print("\n🧮 Null Values Summary:")
print(df.isnull().sum())


✅ CSV saved to: /Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/insurance_hover.csv
📊 Total rows written: 13876

🧮 Null Values Summary:
state_name       0
year             0
quarter          0
district_name    0
metric_type      0
policy_count     0
amount           0
dtype: int64


Into DB

In [20]:
import psycopg2
import pandas as pd

# Load DataFrame
df = pd.read_csv("/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/insurance_hover.csv")

# PostgreSQL Connection Info
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()
print("✅ Connected to PostgreSQL.")

# Optional: Drop & recreate table for fresh import (use with caution)
cur.execute("""
    DROP TABLE IF EXISTS insurance_hover;
    CREATE TABLE insurance_hover (
        id SERIAL PRIMARY KEY,
        state_name TEXT,
        year INTEGER,
        quarter INTEGER,
        district_name TEXT,
        metric_type TEXT,
        policy_count INTEGER,
        amount FLOAT
    );
""")
conn.commit()
print("📦 Table created: insurance_hover")

# Insert rows
for _, row in df.iterrows():
    cur.execute("""
        INSERT INTO insurance_hover (
            state_name, year, quarter, district_name, metric_type, policy_count, amount
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (
        row['state_name'],
        int(row['year']),
        int(row['quarter']),
        row['district_name'],
        row['metric_type'],
        int(row['policy_count']) if not pd.isnull(row['policy_count']) else None,
        float(row['amount']) if not pd.isnull(row['amount']) else None
    ))

conn.commit()
print("✅ All rows inserted successfully.")

# Close connection
cur.close()
conn.close()


✅ Connected to PostgreSQL.
📦 Table created: insurance_hover
✅ All rows inserted successfully.


Map - Transaction

In [45]:
import os
import json
import pandas as pd

base_path = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/pulse/data/map/transaction/hover/country/india/state'
output_csv = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/map_transaction_hover.csv'

trans_data_rows = []

for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    if not os.path.isdir(state_path):
        continue

    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        if not os.path.isdir(year_path):
            continue

        for file in os.listdir(year_path):
            if not file.endswith(".json"):
                continue

            quarter = file.replace(".json", "")
            file_path = os.path.join(year_path, file)

            try:
                with open(file_path, 'r') as f:
                    content = json.load(f)

                hover_list = content.get("data", {}).get("hoverDataList", [])

                for item in hover_list:
                    district = item.get("name")
                    metrics = item.get("metric", [])

                    for metric in metrics:
                        trans_data_rows.append({
                            "state_name": state,
                            "year": int(year),
                            "quarter": int(quarter),
                            "district_name": district,
                            "metric_type": metric.get("type"),
                            "transaction_count": metric.get("count"),
                            "amount": metric.get("amount")
                        })

            except Exception as e:
                print(f"❌ Failed to parse: {file_path} → {e}")

# Create DataFrame
df = pd.DataFrame(trans_data_rows)

# Save to CSV
df.to_csv(output_csv, index=False)
print(f"✅ CSV saved to: {output_csv}")
print("📊 Total rows written:", len(df))

# Null values summary
print("\n🧮 Null Values Summary:")
print(df.isnull().sum())



✅ CSV saved to: /Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/map_transaction_hover.csv
📊 Total rows written: 20604

🧮 Null Values Summary:
state_name           0
year                 0
quarter              0
district_name        0
metric_type          0
transaction_count    0
amount               0
dtype: int64


Into DB

In [21]:
import psycopg2
import pandas as pd

# Read the CSV
csv_file = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/map_transaction_hover.csv"
df = pd.read_csv(csv_file)

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="1234",
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()
    print("✅ Connected to PostgreSQL")

    # Create table if it doesn't exist
    cur.execute("""
        CREATE TABLE IF NOT EXISTS map_transaction_hover (
            state_name TEXT,
            year INTEGER,
            quarter INTEGER,
            district_name TEXT,
            metric_type TEXT,
            transaction_count BIGINT,
            amount DOUBLE PRECISION
        );
    """)
    conn.commit()

    # Insert data row by row
    for _, row in df.iterrows():
        cur.execute("""
            INSERT INTO map_transaction_hover (
                state_name, year, quarter, district_name, metric_type,
                transaction_count, amount
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            row['state_name'], row['year'], row['quarter'],
            row['district_name'], row['metric_type'],
            row['transaction_count'], row['amount']
        ))

    conn.commit()
    print(f"✅ {len(df)} rows inserted into map_transaction_hover")

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    if conn:
        cur.close()
        conn.close()
        print("🔒 PostgreSQL connection closed")


✅ Connected to PostgreSQL
✅ 20604 rows inserted into map_transaction_hover
🔒 PostgreSQL connection closed


Map-hover-user

In [48]:
import os
import json
import pandas as pd

# Input and output paths
base_path = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/pulse/data/map/user/hover/country/india/state'
output_csv = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/map_user.csv'

user_data_rows = []

for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    if not os.path.isdir(state_path):
        continue

    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        if not os.path.isdir(year_path):
            continue

        for file in os.listdir(year_path):
            if not file.endswith(".json"):
                continue

            quarter = file.replace(".json", "")
            file_path = os.path.join(year_path, file)

            with open(file_path, 'r') as f:
                try:
                    data = json.load(f)
                    hover_data = data.get("data", {}).get("hoverData", {})

                    for district, metrics in hover_data.items():
                        user_data_rows.append({
                            "state_name": state,
                            "year": int(year),
                            "quarter": int(quarter),
                            "district_name": district,
                            "registered_users": metrics.get("registeredUsers"),
                            "app_opens": metrics.get("appOpens")
                        })

                except Exception as e:
                    print(f"❌ Error parsing {file_path}: {e}")

# Create DataFrame and save to CSV
df = pd.DataFrame(user_data_rows)
df.to_csv(output_csv, index=False)
print(f"✅ Data saved to {output_csv}")

# 🔍 Null Values Summary
print("\n🧮 Null Values Summary:")
print(df.isnull().sum())


✅ Data saved to /Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/map_user.csv

🧮 Null Values Summary:
state_name          0
year                0
quarter             0
district_name       0
registered_users    0
app_opens           0
dtype: int64


Into DB

In [22]:
import psycopg2
import pandas as pd

# Load CSV
csv_file = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/map_user.csv'
df = pd.read_csv(csv_file)

# PostgreSQL connection details
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()
print("✅ Connected to PostgreSQL.")

# Create table
cur.execute("""
    CREATE TABLE IF NOT EXISTS map_user (
        id SERIAL PRIMARY KEY,
        state_name TEXT,
        year INT,
        quarter INT,
        district_name TEXT,
        registered_users BIGINT,
        app_opens BIGINT
    );
""")
conn.commit()
print("📦 Table 'map_user' ready.")

# Insert data
for _, row in df.iterrows():
    cur.execute("""
        INSERT INTO map_user (
            state_name, year, quarter, district_name, registered_users, app_opens
        ) VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        row['state_name'],
        int(row['year']),
        int(row['quarter']),
        row['district_name'],
        int(row['registered_users']) if pd.notnull(row['registered_users']) else None,
        int(row['app_opens']) if pd.notnull(row['app_opens']) else None
    ))

conn.commit()
cur.close()
conn.close()
print("✅ Data inserted into 'map_user' table.")


✅ Connected to PostgreSQL.
📦 Table 'map_user' ready.
✅ Data inserted into 'map_user' table.


Top - Insurance

In [55]:
pip install pgeocode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [pgeocode]
Note: you may need to restart the kernel to use updated packages.


In [56]:
pip install pincode-india

ERROR: Could not find a version that satisfies the requirement pincode-india (from versions: none)
ERROR: No matching distribution found for pincode-india
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import json
import pandas as pd
import ssl
import certifi
import urllib.request
from io import StringIO

# # ✅ Step 0: Fetch IN.tsv securely and build lookup dictionary
# ssl_context = ssl.create_default_context(cafile=certifi.where())
# url = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/GeoData/IN.txt'
# response = urllib.request.urlopen(url, context=ssl_context)
# in_txt_data = response.read().decode("utf-8")

# in_df = pd.read_csv(StringIO(in_txt_data), sep="\t", dtype={"postal_code": str})
# pincode_to_area = dict(zip(in_df['postal_code'], in_df['place_name']))

# ✅ Function to get area name from pincode

# ✅ Step 0: Load IN.txt locally and build lookup dictionary

local_path = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/GeoData/IN.txt'

# Load the file
in_df = pd.read_csv(local_path, sep="\t", dtype={"postal_code": str})

# Create a simple pincode to area mapping
pincode_to_area = dict(zip(in_df['postal_code'], in_df['place_name']))

# Optional: Full info mapping (place, state, lat/lon)
pincode_full_info = {
    row["postal_code"]: {
        "place_name": row["place_name"],
        "state_name": row["state_name"],
        "latitude": row["latitude"],
        "longitude": row["longitude"]
    }
    for _, row in in_df.iterrows()
}

def get_area_name(pincode):
    try:
        if pd.isna(pincode) or pincode == '':
            return None
        return pincode_to_area.get(str(pincode))
    except:
        return None

# ✅ Base paths
base_path = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/pulse/data/top/insurance/country/india/state'
output_csv_path = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/top_insurance_with_area.csv'

# ✅ Step 1: Extract data from JSON files
insurance_rows = []

for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    if not os.path.isdir(state_path):
        continue

    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        if not os.path.isdir(year_path):
            continue

        for file in os.listdir(year_path):
            if not file.endswith(".json"):
                continue

            quarter = file.replace(".json", "")
            file_path = os.path.join(year_path, file)

            with open(file_path, "r") as f:
                data = json.load(f)

            districts = data.get("data", {}).get("districts", [])
            pincodes = data.get("data", {}).get("pincodes", [])

            # Districts block
            for district_data in districts:
                insurance_rows.append({
                    "state_name": state,
                    "year": int(year),
                    "quarter": int(quarter),
                    "district_name": district_data.get("entityName"),
                    "pincode": None,
                    "metric_type": district_data.get("metric", {}).get("type"),
                    "policy_count": district_data.get("metric", {}).get("count"),
                    "amount": district_data.get("metric", {}).get("amount")
                })

            # Pincodes block
            for pincode_data in pincodes:
                insurance_rows.append({
                    "state_name": state,
                    "year": int(year),
                    "quarter": int(quarter),
                    "district_name": None,
                    "pincode": pincode_data.get("entityName"),
                    "metric_type": pincode_data.get("metric", {}).get("type"),
                    "policy_count": pincode_data.get("metric", {}).get("count"),
                    "amount": pincode_data.get("metric", {}).get("amount")
                })

# ✅ Step 2: Create DataFrame
df = pd.DataFrame(insurance_rows)

# ✅ Step 3: Add area_name using pincode
df['area_name'] = df['pincode'].apply(get_area_name)

# ✅ Step 4: Reorder columns → area_name next to district_name
cols = df.columns.tolist()
district_index = cols.index("district_name")
cols.remove("area_name")
cols.insert(district_index + 1, "area_name")
df = df[cols]

# ✅ Step 5: Save to CSV
df.to_csv(output_csv_path, index=False)
print(f"\n✅ {len(df)} records exported to → {output_csv_path}")

# ✅ Step 6: Null value summary
print("\n🧮 Null Values Summary:")
print(df.isnull().sum())


KeyError: 'postal_code'

In [12]:
import os
import json
import pandas as pd

# ✅ Step 0: Load all_india_pincodes.csv and build lookup dictionaries
pincode_csv_path = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/GeoData/all_india_pincodes.csv'
pin_df = pd.read_csv(pincode_csv_path, dtype={"pincode": str})

# Pincode → Office Name (Area Name)
pincode_to_area = dict(zip(pin_df['pincode'], pin_df['office_name']))

# Optional: Full info dictionary
pincode_full_info = {
    row["pincode"]: {
        "office_name": row["office_name"],
        "district": row["district"],
        "state": row["state"],
        "lat": row["lat"],
        "lon": row["lon"]
    }
    for _, row in pin_df.iterrows()
}

# ✅ Function to get area name from pincode
def get_area_name(pincode):
    try:
        if pd.isna(pincode) or pincode == '':
            return None
        return pincode_to_area.get(str(pincode))
    except:
        return None

# ✅ Paths
base_path = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/pulse/data/top/insurance/country/india/state'
output_csv_path = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/top_insurance_with_area.csv'

# ✅ Step 1: Traverse directories and extract data
insurance_rows = []

for state in os.listdir(base_path):
    state_path = os.path.join(base_path, state)
    if not os.path.isdir(state_path):
        continue

    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        if not os.path.isdir(year_path):
            continue

        for file in os.listdir(year_path):
            if not file.endswith(".json"):
                continue

            quarter = file.replace(".json", "")
            file_path = os.path.join(year_path, file)

            with open(file_path, "r") as f:
                data = json.load(f)

            districts = data.get("data", {}).get("districts", [])
            pincodes = data.get("data", {}).get("pincodes", [])

            # Districts block
            for district_data in districts:
                insurance_rows.append({
                    "state_name": state,
                    "year": int(year),
                    "quarter": int(quarter),
                    "district_name": district_data.get("entityName"),
                    "pincode": None,
                    "metric_type": district_data.get("metric", {}).get("type"),
                    "policy_count": district_data.get("metric", {}).get("count"),
                    "amount": district_data.get("metric", {}).get("amount")
                })

            # Pincodes block
            for pincode_data in pincodes:
                insurance_rows.append({
                    "state_name": state,
                    "year": int(year),
                    "quarter": int(quarter),
                    "district_name": None,
                    "pincode": pincode_data.get("entityName"),
                    "metric_type": pincode_data.get("metric", {}).get("type"),
                    "policy_count": pincode_data.get("metric", {}).get("count"),
                    "amount": pincode_data.get("metric", {}).get("amount")
                })

# ✅ Step 2: Convert to DataFrame
df = pd.DataFrame(insurance_rows)

# ✅ Step 3: Add area_name from pincode
df['area_name'] = df['pincode'].apply(get_area_name)

# ✅ Step 4: Reorder columns
cols = df.columns.tolist()
district_index = cols.index("district_name")
cols.remove("area_name")
cols.insert(district_index + 1, "area_name")
df = df[cols]

# ✅ Step 5: Export to CSV
df.to_csv(output_csv_path, index=False)
print(f"\n✅ {len(df)} records exported to → {output_csv_path}")

# ✅ Step 6: Null value summary
print("\n🧮 Null Values Summary:")
print(df.isnull().sum())



✅ 12276 records exported to → /Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/top_insurance_with_area.csv

🧮 Null Values Summary:
state_name          0
year                0
quarter             0
district_name    6668
area_name        5683
pincode          5611
metric_type         0
policy_count        0
amount              0
dtype: int64


In [ ]:
# Re-edited the IN.txt file 

import pandas as pd

# Load and inspect the raw GeoNames IN.txt structure (manually confirmed earlier)
columns = [
    "country_code", "postal_code", "place_name", "state_name", "state_code",
    "county_name", "county_code", "community_name", "community_code",
    "latitude", "longitude", "accuracy"
]

# Load the cleaned IN.txt file as DataFrame
df = pd.read_csv(
    "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/GeoData/IN.txt",
    sep="\t", header=None, names=columns, dtype={"postal_code": str}
)

# Filter only necessary columns and drop duplicates or missing postal codes
df_clean = df[["postal_code", "place_name", "county_name", "state_name", "latitude", "longitude"]]
df_clean = df_clean.dropna(subset=["postal_code"]).drop_duplicates(subset=["postal_code"])

# Rename for consistency
df_clean.columns = ["pincode", "office_name", "district", "state", "lat", "lon"]

# Save to a cleaned CSV
output_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/GeoData/all_india_pincodes.csv"
df_clean.to_csv(output_path, index=False)

output_path


'/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/GeoData/all_india_pincodes.csv'

In [9]:
import pandas as pd

# Re-define columns since the state was reset
columns = [
    "country_code", "postal_code", "place_name", "state_name", "state_code",
    "county_name", "county_code", "community_name", "community_code",
    "latitude", "longitude", "accuracy"
]

# Load the file again
file_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/GeoData/IN.txt"

# Attempt to load and clean data
try:
    df = pd.read_csv(
        file_path,
        sep="\t", header=None, names=columns, dtype={"postal_code": str}
    )

    df_clean = df[["postal_code", "place_name", "county_name", "state_name", "latitude", "longitude"]]
    df_clean = df_clean.dropna(subset=["postal_code"]).drop_duplicates(subset=["postal_code"])

    # Rename columns
    df_clean.columns = ["pincode", "office_name", "district", "state", "lat", "lon"]

    # Save as downloadable CSV
    output_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/GeoData/all_india_pincodes.csv"
    df_clean.to_csv(output_path, index=False)

    output_path

except Exception as e:
    str(e)


Into DB


In [24]:
import psycopg2
import pandas as pd

# ✅ Step 1: Load the CSV
csv_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/top_insurance_with_area.csv"
df = pd.read_csv(csv_path)

# ✅ Step 2: Connect to PostgreSQL
try:
    conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="1234",
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()
    print("✅ Connected to PostgreSQL.")
except Exception as e:
    print("❌ Connection error:", e)
    exit()

# ✅ Step 3: Create table if not exists
create_table_query = """
CREATE TABLE IF NOT EXISTS insurance_data (
    state_name TEXT,
    year INTEGER,
    quarter INTEGER,
    district_name TEXT,
    area_name TEXT,
    pincode TEXT,
    metric_type TEXT,
    policy_count BIGINT,
    amount DOUBLE PRECISION
);
"""
cur.execute(create_table_query)
conn.commit()

# ✅ Step 4: Insert DataFrame into the table
from io import StringIO

buffer = StringIO()
df.to_csv(buffer, index=False, header=False)
buffer.seek(0)

try:
    cur.copy_expert("""
        COPY insurance_data (state_name, year, quarter, district_name, area_name, pincode, metric_type, policy_count, amount)
        FROM STDIN WITH CSV
    """, buffer)
    conn.commit()
    print(f"✅ {len(df)} rows inserted into 'insurance_data'")
except Exception as e:
    conn.rollback()
    print("❌ Insert error:", e)

# ✅ Step 5: Done — close connection
cur.close()
conn.close()


✅ Connected to PostgreSQL.
✅ 12276 rows inserted into 'insurance_data'


Top - Transcations

In [16]:
import os
import json
import pandas as pd

folder_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/pulse/data/top/transaction/country/india/state"
data = []

for state in os.listdir(folder_path):
    state_path = os.path.join(folder_path, state)
    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        for file in os.listdir(year_path):
            if file.endswith('.json'):
                quarter = int(file.replace('.json', ''))
                with open(os.path.join(year_path, file), 'r') as f:
                    content = json.load(f)
                    try:
                        districts = content['data']['districts']
                        for district in districts:
                            data.append({
                                "state_name": state,
                                "year": int(year),
                                "quarter": quarter,
                                "district_name": district["entityName"],
                                "transaction_count": district["metric"]["count"],
                                "transaction_amount": district["metric"]["amount"]
                            })
                    except Exception as e:
                        print(f"⚠️ Skipping file: {file}, Error: {e}")

df = pd.DataFrame(data)
output_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/top_transaction.csv"
df.to_csv(output_path, index=False)
print(f"✅ CSV exported successfully to:\n{output_path}")

print("\n🧮 Null Values Summary:")
print(df.isnull().sum())


✅ CSV exported successfully to:
/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/top_transaction.csv

🧮 Null Values Summary:
state_name            0
year                  0
quarter               0
district_name         0
transaction_count     0
transaction_amount    0
dtype: int64


In [18]:
print(len(data))

8296


Into DB


In [25]:
import psycopg2
import pandas as pd

# Step 1: Load CSV
csv_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/top_transaction.csv"
df = pd.read_csv(csv_path)

# Step 2: Connect to PostgreSQL
try:
    conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="1234",
        host="localhost",
        port="5432"
    )
    cursor = conn.cursor()
    print("✅ Connected to PostgreSQL.")
except Exception as e:
    print("❌ Connection failed:", e)
    exit()

# Step 3: Create Table
create_query = """
CREATE TABLE IF NOT EXISTS top_transaction (
    id SERIAL PRIMARY KEY,
    state_name TEXT,
    year INT,
    quarter INT,
    district_name TEXT,
    transaction_count BIGINT,
    transaction_amount DOUBLE PRECISION
);
"""
cursor.execute(create_query)
conn.commit()

# Step 4: Insert Data
for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO top_transaction (
            state_name, year, quarter, district_name, transaction_count, transaction_amount
        ) VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        row['state_name'],
        int(row['year']),
        int(row['quarter']),
        row['district_name'],
        int(row['transaction_count']),
        float(row['transaction_amount'])
    ))

conn.commit()
print("✅ Data inserted successfully.")

# Step 5: Close
cursor.close()
conn.close()


✅ Connected to PostgreSQL.
✅ Data inserted successfully.


Top -User 

In [13]:
import os
import json
import pandas as pd

# 📂 Paths
json_folder = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/pulse/data/top/user/country/india/state"
csv_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/top_user_by_pincode.csv"
pincode_csv_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/GeoData/all_india_pincodes.csv"

# ✅ Step 0: Load all_india_pincodes.csv and build lookup dictionary
pin_df = pd.read_csv(pincode_csv_path, dtype={"pincode": str})

pincode_info_map = {
    row["pincode"]: {
        "area_name": row["office_name"],
        "district": row["district"],
        "state": row["state"],
        "lat": row.get("lat"),
        "lon": row.get("lon")
    }
    for _, row in pin_df.iterrows()
}

def get_pincode_info(pincode):
    if pd.isna(pincode) or pincode == '':
        return {}
    return pincode_info_map.get(str(pincode), {})

# 📝 Records list
records = []

# 🔁 Traverse states > years > quarters
for state in os.listdir(json_folder):
    state_path = os.path.join(json_folder, state)
    if not os.path.isdir(state_path):
        continue

    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        if not os.path.isdir(year_path):
            continue

        for quarter_file in os.listdir(year_path):
            if not quarter_file.endswith(".json"):
                continue

            file_path = os.path.join(year_path, quarter_file)
            with open(file_path, "r") as f:
                try:
                    data = json.load(f)
                    state_name_from_path = state.replace("-", " ").title()

                    pincodes = data.get("data", {}).get("pincodes", [])
                    for pincode_obj in pincodes:
                        pincode = str(pincode_obj.get("name"))
                        registeredUsers = pincode_obj.get("registeredUsers")

                        info = get_pincode_info(pincode)
                        records.append({
                            "state": info.get("state", state_name_from_path),
                            "district_name": info.get("district"),
                            "area_name": info.get("area_name"),
                            "pincode": pincode,
                            "registeredUsers": registeredUsers,
                            # "appOpens": None
                        })
                except json.JSONDecodeError:
                    print(f"❌ Failed to parse {file_path}")

# 📊 Save to CSV
df = pd.DataFrame(records)
df.to_csv(csv_path, index=False)
print(f"✅ CSV saved to: {csv_path}")

# 🧮 Null Values Summary
print("\n🧮 Null Values Summary:")
print(df.isnull().sum())


✅ CSV saved to: /Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/top_user_by_pincode.csv

🧮 Null Values Summary:
state               0
district_name      70
area_name          70
pincode             0
registeredUsers     0
dtype: int64


Into DB

In [14]:
import pandas as pd
import psycopg2
from psycopg2 import sql

# Load the CSV
csv_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/cleaned_csv/top_user_by_pincode.csv"
df = pd.read_csv(csv_path)

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="1234",
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()
    print("✅ Connected to PostgreSQL.")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    raise

# Create table if it doesn't exist
create_table_query = """
CREATE TABLE IF NOT EXISTS top_user_by_pincode (
    id SERIAL PRIMARY KEY,
    state TEXT,
    district_name TEXT,
    area_name TEXT,
    pincode TEXT,
    registeredUsers INTEGER
);
"""
cur.execute(create_table_query)
conn.commit()
print("✅ Table is ready.")

# Insert records
insert_query = """
INSERT INTO top_user_by_pincode (state, district_name, area_name, pincode, registeredUsers)
VALUES (%s, %s, %s, %s, %s)
"""

for _, row in df.iterrows():
    cur.execute(insert_query, (
        row["state"],
        row["district_name"],
        row["area_name"],
        row["pincode"],
        int(row["registeredUsers"]) if not pd.isna(row["registeredUsers"]) else None
    ))

conn.commit()
print(f"✅ {len(df)} records inserted into 'top_user_by_pincode'.")

# Close connection
cur.close()
conn.close()
print("🔒 Connection closed.")


✅ Connected to PostgreSQL.
✅ Table is ready.
✅ 10000 records inserted into 'top_user_by_pincode'.
🔒 Connection closed.


In [13]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 4.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 3.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22/22 [streamlit]22 [streamlit]]
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install matplotlib plotly 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 3.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 3.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [matplotlib]7 [matplotlib]
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install watchdog

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install pydeck pandas


Note: you may need to restart the kernel to use updated packages.


In [22]:
import os
import json
import pandas as pd

# === 📂 Input Paths ===
json_folder = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/pulse/data/map/transaction/hover/country/india/state"
latlon_csv_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/GeoData/all_india_pincodes.csv"

# === 💾 Output Path ===
output_csv_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover.csv"

# === 📍 Load Latitude & Longitude Data ===
latlon_df = pd.read_csv(latlon_csv_path)

# Normalize district names
latlon_df['district_normalized'] = latlon_df['district'].str.strip().str.lower()

# Build map from normalized district → (lat, lon)
latlon_map = latlon_df.groupby('district_normalized')[['lat', 'lon']].first().to_dict(orient='index')

def get_lat_lon(district_name):
    if not district_name:
        return (None, None)
    normalized = district_name.strip().lower()
    if normalized in latlon_map:
        return latlon_map[normalized]['lat'], latlon_map[normalized]['lon']
    
    # Try partial match (fallback)
    for district_key in latlon_map:
        if normalized in district_key or district_key in normalized:
            return latlon_map[district_key]['lat'], latlon_map[district_key]['lon']
    return (None, None)

# === 📊 Extract Data ===
records = []

for state in os.listdir(json_folder):
    state_path = os.path.join(json_folder, state)
    if not os.path.isdir(state_path):
        continue

    for year in os.listdir(state_path):
        year_path = os.path.join(state_path, year)
        if not os.path.isdir(year_path):
            continue

        for quarter_file in os.listdir(year_path):
            if not quarter_file.endswith(".json"):
                continue

            file_path = os.path.join(year_path, quarter_file)
            with open(file_path, "r") as f:
                try:
                    data = json.load(f)
                    hover_data = data.get("data", {}).get("hoverDataList", [])
                    state_name = state.replace("-", " ").title()

                    for entry in hover_data:
                        district = entry.get("name")
                        metrics = entry.get("metric", [])
                        total_metric = next((m for m in metrics if m.get("type") == "TOTAL"), {})
                        count = total_metric.get("count")
                        amount = total_metric.get("amount")

                        lat, lon = get_lat_lon(district)

                        records.append({
                            "state": state_name,
                            "district": district,
                            "transaction_count": count,
                            "transaction_amount": amount,
                            "latitude": lat,
                            "longitude": lon
                        })

                except json.JSONDecodeError:
                    print(f"Error decoding JSON in file: {file_path}")
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

# === 📤 Save to CSV ===
df = pd.DataFrame(records)
df.to_csv(output_csv_path, index=False)
print(f"✅ CSV saved to: {output_csv_path}")

# === 🧮 Null Values Summary ===
print("\n🧮 Null Values Summary:")
print(df.isnull().sum())


✅ CSV saved to: /Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover.csv

🧮 Null Values Summary:
state                    0
district                 0
transaction_count        0
transaction_amount       0
latitude              5066
longitude             5066
dtype: int64


In [23]:
pip install folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [folium]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import psycopg2

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)

# List columns in the table
df_columns = pd.read_sql("""
    SELECT column_name 
    FROM information_schema.columns 
    WHERE table_name = 'map_transaction_hover'
""", conn)

print(df_columns)


         column_name
0            quarter
1  transaction_count
2             amount
3               year
4      district_name
5         state_name
6        metric_type


/var/folders/f4/l1t8cgjd1v32y4nd2y9gl0580000gn/T/ipykernel_30822/2145687871.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_columns = pd.read_sql("""


In [3]:
import pandas as pd
import plotly.express as px

# Load your processed CSV file (with lat/lon already merged)
df = pd.read_csv("/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover.csv")

# Optional: format amount in Cr
df["formatted_amount"] = df["amount"].apply(lambda x: f"₹{x/1e7:.2f} Cr")

# Basic 2D India map using lat/lon
fig = px.scatter_geo(
    df,
    lat="latitude",
    lon="longitude",
    scope="asia",  # India falls under Asia
    hover_name="district_name",
    hover_data={
        "state_name": True,
        "transaction_count": True,
        "formatted_amount": True,
        "latitude": False,
        "longitude": False
    },
    color="amount",
    size="transaction_count",
    projection="natural earth",
    title="PhonePe Transactions Map (India)"
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(height=600, margin={"r":0,"t":50,"l":0,"b":0})
fig.show()


KeyError: 'amount'

In [5]:
import pandas as pd
import psycopg2

# === Load CSV ===
csv_path = "/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover.csv"
csv_df = pd.read_csv(csv_path)

# Rename CSV columns to match SQL for join
csv_df = csv_df.rename(columns={
    "state": "state_name",
    "district": "district_name"
})

# === Connect to PostgreSQL and fetch year & quarter ===
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
query = """
SELECT DISTINCT state_name, district_name, year, quarter
FROM map_transaction_hover;
"""
sql_df = pd.read_sql(query, conn)
conn.close()

# Drop duplicates for clean merge
sql_df = sql_df.drop_duplicates(subset=["state_name", "district_name"])

# === Merge without overwriting original CSV columns ===
merged_df = pd.merge(csv_df, sql_df, on=["state_name", "district_name"], how="left")

# === Save the updated CSV ===
merged_df.to_csv(csv_path, index=False)

print("✅ Year and Quarter added to CSV without modifying existing data.")

# # === Save back to CSV ===
# merged_df.to_csv(csv_path, index=False)
# print(f"✅ CSV updated with year and quarter and saved to:\n{csv_path}")

# === Optional: Show nulls if any
print("\n🧮 Null Values Summary:")
print(merged_df[["state_name", "district_name", "year", "quarter"]].isnull().sum())


✅ Year and Quarter added to CSV without modifying existing data.

🧮 Null Values Summary:
state_name           0
district_name        0
year             20604
quarter          20604
dtype: int64


/var/folders/f4/l1t8cgjd1v32y4nd2y9gl0580000gn/T/ipykernel_48951/833484456.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_df = pd.read_sql(query, conn)


In [8]:
import pandas as pd
import psycopg2

# Step 1: Load backup CSV
csv_path = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover_2.csv'
df_csv = pd.read_csv(csv_path)

# Step 2: Connect to PostgreSQL and fetch year & quarter
conn = psycopg2.connect(
    dbname='postgres',
    user='postgres',
    password='1234',
    host='localhost',
    port='5432'
)

query = '''
SELECT 
    state_name AS state,
    district_name AS district,
    transaction_count,
    amount AS transaction_amount,
    year,
    quarter
FROM map_transaction_hover
'''

df_db = pd.read_sql(query, conn)
conn.close()

# Step 3: Merge based on state, district, transaction_count, and transaction_amount
merged_df = pd.merge(
    df_csv,
    df_db,
    on=['state', 'district', 'transaction_count', 'transaction_amount'],
    how='left'
)

# Step 4: Save updated CSV
output_path = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover.csv'
merged_df.to_csv(output_path, index=False)

print("✅ Updated CSV with year and quarter saved to:")
print(output_path)

print("\n🧮 Rows missing year/quarter:")
print(merged_df[merged_df['year'].isnull()])


✅ Updated CSV with year and quarter saved to:
/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover.csv

🧮 Rows missing year/quarter:
                           state                           district  \
0      Andaman & Nicobar Islands  north and middle andaman district   
1      Andaman & Nicobar Islands             south andaman district   
2      Andaman & Nicobar Islands                  nicobars district   
3      Andaman & Nicobar Islands  north and middle andaman district   
4      Andaman & Nicobar Islands             south andaman district   
...                          ...                                ...   
20599                   Nagaland                  tuensang district   
20600                   Nagaland                     peren district   
20601                   Nagaland                   dimapur district   
20602                   Nagaland                 zunheboto district   
20603                   Nagaland                  

/var/folders/f4/l1t8cgjd1v32y4nd2y9gl0580000gn/T/ipykernel_48951/3666958161.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_db = pd.read_sql(query, conn)


In [11]:
import pandas as pd
import psycopg2

# Load the CSV
csv_path = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover_2.csv'
df = pd.read_csv(csv_path)

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="1234",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()

# Add columns with null for now
df["latitude"] = None
df["longitude"] = None

# Loop through rows and fetch latitude & longitude from SQL
for idx, row in df.iterrows():
    state = row['state']
    district = row['district']

    cursor.execute("""
        SELECT latitude, longitude
        FROM map_transaction_hover
        WHERE state_name = %s AND district_name = %s;
    """, (state, district))

    result = cursor.fetchone()
    if result:
        lat, lon = result
        df.at[idx, "latitude"] = lat
        df.at[idx, "longitude"] = lon

# Save the updated CSV
df.to_csv(csv_path, index=False)
print(f"✅ Updated CSV saved at: {csv_path}")

# Close DB
cursor.close()
conn.close()


✅ Updated CSV saved at: /Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover_2.csv


In [12]:
import pandas as pd
from sqlalchemy import create_engine
import sys

# --- 1. DEFINE YOUR FILE PATHS AND DATABASE CREDENTIALS ---

# The CSV file containing the latitude and longitude data
CSV_PATH = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover_3.csv'

# Your PostgreSQL database connection details
DB_CONFIG = {
    "user": "postgres",
    "password": "1234",
    "host": "localhost",
    "port": "5432",
    "dbname": "postgres"
}

# The name of the table you want to update
TABLE_NAME = 'map_transaction_hover'

# --- 2. CONNECT TO THE DATABASE AND LOAD THE TABLE ---

try:
    # Create a database connection URL and engine
    db_url = f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"
    engine = create_engine(db_url)
    
    print(f"✅ Successfully connected to the '{DB_CONFIG['dbname']}' database.")
    
    # Load the entire 'map_transaction_hover' table into a DataFrame
    db_df = pd.read_sql_table(TABLE_NAME, engine)
    print(f"✅ Loaded {len(db_df)} rows from the '{TABLE_NAME}' table.")

except Exception as e:
    print(f"❌ Error connecting to the database or loading the table: {e}")
    sys.exit() # Exit the script if connection fails

# --- 3. LOAD THE LATITUDE AND LONGITUDE DATA FROM THE CSV ---

try:
    # Read the CSV file
    geo_df = pd.read_csv(CSV_PATH)
    
    # We only need the key columns ('state', 'district') and the geo data
    # Assumption: Your CSV file has 'state' and 'district' columns that match the database table
    required_csv_cols = ['state', 'district', 'latitude', 'longitude']
    if not all(col in geo_df.columns for col in required_csv_cols):
        print(f"❌ CSV file is missing required columns. Expected: {required_csv_cols}")
        sys.exit()

    geo_df = geo_df[required_csv_cols]
    
    # Remove any duplicate district entries to ensure a clean merge
    geo_df.drop_duplicates(subset=['state', 'district'], inplace=True)
    
    print(f"✅ Loaded {len(geo_df)} unique geo-locations from '{CSV_PATH}'.")

except FileNotFoundError:
    print(f"❌ Error: The file was not found at '{CSV_PATH}'. Please check the path.")
    sys.exit()
except Exception as e:
    print(f"❌ Error reading the CSV file: {e}")
    sys.exit()

# --- 4. MERGE THE TWO DATAFRAMES ---

# We perform a 'left' merge to keep all records from your original database table
# and add the lat/lon where the 'state' and 'district' match.
print("\nMerging the database table with the CSV data...")
updated_df = pd.merge(db_df, geo_df, on=['state', 'district'], how='left')

# Check if any rows failed to find a matching location
unmapped_rows = updated_df['latitude'].isnull().sum()
if unmapped_rows > 0:
    print(f"⚠️ Warning: {unmapped_rows} rows in your database could not find a matching latitude/longitude.")
else:
    print("✅ All rows were successfully mapped with geographic coordinates.")

# --- 5. WRITE THE UPDATED DATAFRAME BACK TO THE DATABASE ---

try:
    print(f"\nWriting the updated data back to the '{TABLE_NAME}' table...")
    
    # This will DROP the old table and CREATE a new one with the merged data.
    updated_df.to_sql(
        TABLE_NAME,
        engine,
        if_exists='replace', # This replaces the entire table
        index=False          # Do not write the DataFrame index as a column
    )
    
    print("\n🎉 Success! Your database table has been updated with latitude and longitude values.")

except Exception as e:
    print(f"❌ An error occurred while writing to the database: {e}")

✅ Successfully connected to the 'postgres' database.
✅ Loaded 20604 rows from the 'map_transaction_hover' table.
✅ Loaded 852 unique geo-locations from '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover_3.csv'.

Merging the database table with the CSV data...


KeyError: 'state'

In [13]:
import pandas as pd
from sqlalchemy import create_engine
import sys

# --- Your file paths and database credentials ---
CSV_PATH = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover_3.csv'
DB_CONFIG = {
    "user": "postgres", "password": "1234", "host": "localhost",
    "port": "5432", "dbname": "postgres"
}
TABLE_NAME = 'map_transaction_hover'

# --- Load data and print columns ---
try:
    # Load from Database
    db_url = f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"
    engine = create_engine(db_url)
    db_df = pd.read_sql_table(TABLE_NAME, engine)
    print("--- 🧐 Columns in your Database Table ---")
    print(db_df.columns.tolist()) # This prints the DB column names

    # Load from CSV
    geo_df = pd.read_csv(CSV_PATH)
    print("\n--- 🧐 Columns in your CSV File ---")
    print(geo_df.columns.tolist()) # This prints the CSV column names

except Exception as e:
    print(f"❌ An error occurred: {e}")

--- 🧐 Columns in your Database Table ---
['state_name', 'year', 'quarter', 'district_name', 'metric_type', 'transaction_count', 'amount', 'latitude', 'longitude']

--- 🧐 Columns in your CSV File ---
['state', 'district', 'transaction_count', 'transaction_amount', 'latitude', 'longitude']


In [1]:
import pandas as pd
from sqlalchemy import create_engine
import sys

# --- 1. DEFINE YOUR FILE PATHS AND DATABASE CREDENTIALS ---
CSV_PATH = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover_3.csv'
DB_CONFIG = {
    "user": "postgres", "password": "1234", "host": "localhost",
    "port": "5432", "dbname": "postgres"
}
TABLE_NAME = 'map_transaction_hover'

# --- 2. CONNECT TO THE DATABASE AND LOAD THE TABLE ---
try:
    db_url = f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"
    engine = create_engine(db_url)
    db_df = pd.read_sql_table(TABLE_NAME, engine)
    print(f"✅ Loaded {len(db_df)} rows from the '{TABLE_NAME}' table.")
except Exception as e:
    print(f"❌ Error connecting to the database: {e}")
    sys.exit()

# --- 3. LOAD THE LATITUDE AND LONGITUDE DATA FROM THE CSV ---
try:
    geo_df = pd.read_csv(CSV_PATH)
    geo_df = geo_df[['state', 'district', 'latitude', 'longitude']]
    geo_df.drop_duplicates(subset=['state', 'district'], inplace=True)
    print(f"✅ Loaded {len(geo_df)} unique geo-locations from '{CSV_PATH}'.")
except Exception as e:
    print(f"❌ Error reading or processing the CSV file: {e}")
    sys.exit()

# ⭐ --- 4. NEW: NORMALIZE JOIN KEYS FOR ACCURATE MERGING ---
print("\nCleaning and standardizing state and district names for a reliable merge...")

# Clean the database DataFrame columns
for col in ['state_name', 'district_name']:
    db_df[col] = db_df[col].str.lower().str.strip()

# Clean the CSV DataFrame columns
for col in ['state', 'district']:
    geo_df[col] = geo_df[col].str.lower().str.strip()

print("✅ Join keys have been normalized.")

# --- 5. MERGE THE TWO DATAFRAMES ---
print("\nMerging the database table with the CSV data...")

db_df_without_geo = db_df.drop(columns=['latitude', 'longitude'], errors='ignore')

updated_df = pd.merge(
    db_df_without_geo,
    geo_df,
    left_on=['state_name', 'district_name'],
    right_on=['state', 'district'],
    how='left'
)

updated_df.drop(columns=['state', 'district'], inplace=True, errors='ignore')

unmapped_rows = updated_df['latitude'].isnull().sum()
if unmapped_rows > 0:
    print(f"⚠️ Warning: {unmapped_rows} of {len(updated_df)} rows could not find a matching location.")
else:
    print("✅ All rows were successfully mapped with geographic coordinates.")

# --- 6. WRITE THE UPDATED DATAFRAME BACK TO THE DATABASE ---
try:
    print(f"\nWriting updated data back to '{TABLE_NAME}'...")
    updated_df.to_sql(TABLE_NAME, engine, if_exists='replace', index=False)
    print("\n🎉 Success! Your database table has been updated.")
except Exception as e:
    print(f"❌ An error occurred while writing to the database: {e}")

✅ Loaded 20604 rows from the 'map_transaction_hover' table.
✅ Loaded 852 unique geo-locations from '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover_3.csv'.

Cleaning and standardizing state and district names for a reliable merge...
✅ Join keys have been normalized.

Merging the database table with the CSV data...
⚠️ Warning: 10764 of 20604 rows could not find a matching location.

Writing updated data back to 'map_transaction_hover'...

🎉 Success! Your database table has been updated.


In [22]:
import pandas as pd
from sqlalchemy import create_engine
import sys

# --- 1. CONFIGURATION ---
CSV_PATH = '/Users/macbook/Desktop/DS_Project/Phone_pe_pluse/PhonePe_Dashboard/map_transaction_hover_3.csv'
DB_CONFIG = {
    "user": "postgres", "password": "1234", "host": "localhost",
    "port": "5432", "dbname": "postgres"
}
TABLE_NAME = 'map_transaction_hover'
OUTPUT_FILE = 'unmapped_districts.csv' # The file this script will create

# --- 2. LOAD AND CLEAN DATA ---
try:
    # Load from Database
    db_url = f"postgresql+psycopg2://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['dbname']}"
    engine = create_engine(db_url)
    db_df = pd.read_sql_table(TABLE_NAME, engine)

    # Load from CSV
    geo_df = pd.read_csv(CSV_PATH)
    geo_df = geo_df[['state', 'district', 'latitude', 'longitude']]

    # Normalize join keys
    for col in ['state_name', 'district_name']:
        db_df[col] = db_df[col].str.lower().str.strip()
    for col in ['state', 'district']:
        geo_df[col] = geo_df[col].str.lower().str.strip()

except Exception as e:
    print(f"❌ An error occurred during file loading: {e}")
    sys.exit()

# --- 3. PERFORM MERGE TO FIND MISMATCHES ---
merged_df = pd.merge(
    db_df,
    geo_df,
    left_on=['state_name', 'district_name'],
    right_on=['state', 'district'],
    how='left',
    indicator=True # Adds a column to show the source of the match
)

# Filter for rows that are only in the database (no match in CSV)
unmapped_df = merged_df[merged_df['_merge'] == 'left_only']

# --- 4. SAVE THE UNIQUE UNMAPPED NAMES TO A CSV FILE ---
if not unmapped_df.empty:
    # Get the unique list of unmapped state/district pairs
    unique_unmapped = unmapped_df[['state_name', 'district_name']].drop_duplicates().sort_values(by=['state_name', 'district_name'])
    
    # Save the list to a new CSV file
    unique_unmapped.to_csv(OUTPUT_FILE, index=False)
    
    print(f"⚠️ Found {len(unique_unmapped)} unique state/district pairs that could not be mapped.")
    print(f"✅ A list of these has been saved to '{OUTPUT_FILE}' for you to review.")
else:
    print("🎉 Great news! All rows were mapped successfully.")

⚠️ Found 302 unique state/district pairs that could not be mapped.
✅ A list of these has been saved to 'unmapped_districts.csv' for you to review.
